In [2]:
import pandas as pd
import gensim
import pickle
import os
import re
import gensim

load in and preprocess dataframes for each perspective.

In [ ]:
os.chdir('/home/jovyan/work/models_storage/preprocessed_texts')
def open_test_data(path):
    return open(path, 'rb')
with open_test_data('training_relational_preprocessed_100321.pkl') as f:
    rela = pickle.load(f)

with open_test_data('training_demographic_preprocessed_100321.pkl') as f:
    demog = pickle.load(f)

with open_test_data('training_orgs_preprocessed_100321.pkl') as f:
    orgs = pickle.load(f)

with open_test_data('training_cultural_preprocessed_100321.pkl') as f:
    cult = pickle.load(f)
    

import itertools
full_text = []

for i in cult['text']:
    joined = list(itertools.chain(*i))
    full_text.append(" ".join(joined))

full_text_demog = []
for i in demog['text']:
    joined = list(itertools.chain(*i))
    full_text_demog.append(" ".join(joined))

full_text_orgs = []
for j in orgs['text']:
    joined = list(itertools.chain(*j))
    full_text_orgs.append(" ".join(joined))

full_text_rela = []
for j in rela['text']:
    joined = list(itertools.chain(*j))
    full_text_rela.append(" ".join(joined))

cult['full_text'] = full_text
demog['full_text'] = full_text_demog
orgs['full_text'] = full_text_orgs
rela['full_text'] = full_text_rela

def remove_tags(article):
    article = re.sub('<plain_text> <page sequence="1">', '', article)
    article = re.sub(r'</page>(\<.*?\>)', ' \n ', article)
    # xml tags
    article = re.sub(r'<.*?>', '', article)
    article = re.sub(r'<body.*\n\s*.*\s*.*>', '', article)
    return article

tags_removed = [remove_tags(art) for art in cult['full_text']]
tags_removed_demog = [remove_tags(art) for art in demog['full_text']]
tags_removed_org = [remove_tags(art) for art in orgs['full_text']]
tags_removed_rela = [remove_tags(art) for art in rela['full_text']]
cult['text_no_tags'] = tags_removed
demog['text_no_tags'] = tags_removed_demog
orgs['text_no_tags'] = tags_removed_org
rela['text_no_tags'] = tags_removed_rela

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['one', 'two', 'three', 'amp', 'may', 'can', 'new', 'also', 'and'])

import string
import re
import nltk

def word_process(tt):
    """
    helper function to lower text, remove stop words, numbers, and empty 
    """
    
    tt = tt.lower()
    
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~=\n'''
    # Removing punctuations in string 
    # Using loop + punctuation string 


    for ele in tt:  
        if ele in punc:  
            tt = tt.replace(ele, " ")  

    # read tokens
    tokens = tt.split()
    lst = [token.translate(punc).lower() for token in tokens ]
    
    #remove stop words
    filtered = []
    for i in lst:
        if i not in stop_words:
            filtered.append(i)
    
    # removing singular numbers and singular letters
    pattern = '[0-9]'
    filtered = [re.sub(pattern, '', i) for i in filtered] 
    new = []
    for inp in filtered:
        new.append(' '.join( [w for w in inp.split() if len(w)>1] ))
        
    # filter out empty strings 
    new = [i for i in new if i] 

    dt = [d.split() for d in new]
    
  
    st = ''
    for i in dt:
        st = st + " " + i[0]
    return st
    

cult['processed']  =  cult['text_no_tags'].apply(word_process)
rela['processed']  =  rela['text_no_tags'].apply(word_process)
demog['processed']  =  demog['text_no_tags'].apply(word_process)
orgs['processed']  =  orgs['text_no_tags'].apply(word_process)

cult_pos = cult[cult['cultural_score']==1]
cult_neg = cult[cult['cultural_score']==0]

rela_pos = rela[rela['relational_score']==1]
rela_neg = rela[rela['relational_score']==0]

demog_pos = demog[demog['demographic_score']==1]
demog_neg = demog[demog['demographic_score']==0]

pos_cult = cult_pos.sample(20, replace = False)
neg_cult = cult_neg.sample(20, replace = False)

pos_rela = rela_pos.sample(20, replace = False)
neg_rela = rela_neg.sample(20, replace = False)

pos_demog = demog_pos.sample(20, replace = False)
neg_demog = demog_neg.sample(20, replace = False)

pos_cult.to_csv('/home/jovyan/work/nancyxu/pos_cult.csv')
neg_cult.to_csv('/home/jovyan/work/nancyxu/neg_cult.csv')
pos_rela.to_csv('/home/jovyan/work/nancyxu/pos_rela.csv')
neg_rela.to_csv('/home/jovyan/work/nancyxu/neg_rela.csv')
pos_demog.to_csv('/home/jovyan/work/nancyxu/pos_demog.csv')
neg_demog.to_csv('/home/jovyan/work/nancyxu/neg_demog.csv')


In [5]:

pos_cult = pd.read_csv('pos_cult.csv')
neg_cult = pd.read_csv('neg_cult.csv')
pos_rela = pd.read_csv('pos_rela.csv')
neg_rela = pd.read_csv('neg_rela.csv')
pos_demog = pd.read_csv('pos_demog.csv')
neg_demog = pd.read_csv('neg_demog.csv')


In [ ]:
## get dataframe with all articles belong to the 3 perspectives 
df = pd.concat([cult_pos,rela_pos,demog_pos])

train w2v model on the positively labeled articles for each perspective.

In [13]:
w2v = gensim.models.Word2Vec(df['processed'])

In [14]:
def calc_wmd(a,b):
    return w2v.wmdistance(a,b)

In [16]:
# import numpy as np
# def find_mean_and_closest_article(article,df,topk):
#     """
#     calcalate the mean wmd of a certain article from other articles in the same perspective,
#     then give the top k closest articles in the same perspective.
#     """
#     d = []
#     for i in np.arange(len(df)):
#         d.append(calc_wmd(article,df.loc[i,'processed']))
#     dis_dict = pd.DataFrame(list(zip(d,df['processed'])), columns = ['distance', 'article'])
#     dis_dict['full_text'] = df['full_text']
#     dis_dict['edited_filename'] = df['edited_filename'] 
#     dis_dict = dis_dict[dis_dict['article']!=article]
#     srted = dis_dict.sort_values('distance').head(topk)

#     return np.mean(dis_dict['distance']), tuple(zip(srted['edited_filename'], srted['distance']))
        
# def get_cols_sims(df):
#     """
#     get mean wmd similarity scores 
#     """
#     results = pos_cult['processed'].apply(lambda x: find_closest_article(x,pos_cult,10))
#     mean_sims_score = [i[0] for i in results]
#     top10_sims_articles = [i[1] for i in results]
#     df['mean_sims_score'] = mean_sims_score
#     df['top10_sims_articles'] = top10_sims_articles
#     return df.sort_values('mean_sims_score')

### read in core seed terms to compute WMD between an article and core terms

reference: https://radimrehurek.com/gensim/auto_examples/tutorials/run_wmd.html

In [17]:
def calc_wmd(a,b):
    return w2v.wmdistance(a,b)

In [18]:
def read_txt_lst(file):
    txt_file = open(file, "r")
    file_content = txt_file.read()
    # print("The file content are: ", file_content)

    content_list = file_content.split("\n")
    txt_file.close()
    return content_list

In [19]:
dem_core=read_txt_lst('demographic_core.txt')
rela_core=read_txt_lst('relational_core.txt')
cult_core = read_txt_lst('cultural_core.txt')

In [20]:
def get_sims_with_perspective(article):
  """
  get wmd similarity between this article and the core terms of each perspective
  """

    dem_score = calc_wmd(article,' '.join(dem_core))
    rela_score = calc_wmd(article,' '.join(rela_core))
    cult_score = calc_wmd(article,' '.join(cult_core))
    return (dem_score,rela_score,cult_score)

In [24]:
def get_sim_perspective(df):

    """
    returns modified df with extra columns dem_score,rela_score,cult_score
    denoting the wmd score calculated for each article.
    """

    results=df['processed'].apply(lambda x:get_sims_with_perspective(x))
    dem_scores = [i[0] for i in results]
    rela_scores = [i[1] for i in results]
    cult_scores = [i[2] for i in results]
    df['dem_scores'] = dem_scores
    df['rela_scores'] = rela_scores
    df['cult_scores']= cult_scores
    return df

In [22]:
pos_cult = get_sim_perspective(pos_cult)
neg_cult = get_sim_perspective(neg_cult)

pos_demog = get_sim_perspective(pos_demog)
neg_demog = get_sim_perspective(neg_demog)

pos_rela = get_sim_perspective(pos_rela)
neg_rela = get_sim_perspective(neg_rela)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).
  


In [28]:
pos_cult

,Unnamed: 0,text,cultural_score,primary_subject,edited_filename,article_name,full_text,text_no_tags,processed,dem_scores,rela_scores,cult_scores
0,691,"[['Institutional', 'Sources', 'of', 'Practice'...",1.0,Management & Organizational Behavior,10.2307_2667124,Institutional Sources of Practice Variation: S...,Institutional Sources of Practice Variation: S...,Institutional Sources of Practice Variation: S...,institutional sources practice variation staf...,0.349953,0.649178,0.640662
1,119,"[['Autonomy', 'and', 'the', ""Commercializat'io...",1.0,Sociology,10.2307_3518065,Autonomy and the Commercialization of Higher E...,Autonomy and the Commercializat'ion of Higher ...,Autonomy and the Commercializat'ion of Higher ...,autonomy commercializat ion higher education ...,0.388643,0.627294,0.669451
2,570,"[['Bringing', 'Managers', 'into', 'Theories', ...",1.0,Management & Organizational Behavior,10.2307_4135118,Bringing Managers into Theories of Multimarket...,Bringing Managers into Theories of Multimarket...,Bringing Managers into Theories of Multimarket...,bringing managers theories multimarket compet...,0.428987,0.614230,0.714756
3,659,"[['THE', 'DECLINE', 'AND', 'FALL', 'OF', 'THE'...",1.0,Sociology,10.2307_2095931,The Decline and Fall of the Conglomerate Firm ...,THE DECLINE AND FALL OF THE CONGLOMERATE FIRM ...,THE DECLINE AND FALL OF THE CONGLOMERATE FIRM ...,decline fall conglomerate firm deinstitutiona...,0.435695,0.649773,0.637447
4,880,"[['THE', 'ROLE', 'OF', 'INSTITUTIONALIZATION',...",1.0,Sociology,10.2307_2094862,The Role of Institutionalization in Cultural P...,THE ROLE OF INSTITUTIONALIZATION IN CULTURAL P...,THE ROLE OF INSTITUTIONALIZATION IN CULTURAL P...,role institutionalization cultural persistenc...,0.384446,0.587483,0.631942
5,113,"[['Institutional', 'Isomorphism', 'and', 'Info...",1.0,Sociology,10.2307_3096861,Institutional Isomorphism and Informal Social ...,Institutional Isomorphism and Informal Social ...,Institutional Isomorphism and Informal Social ...,institutional isomorphism informal social con...,0.421624,0.638911,0.635368
6,689,"[['Organization', 'inf""', 'Vol.', 'No.', 'Janu...",1.0,Management & Organizational Behavior,10.2307_27765958,Young and No Money? Never Mind: The Material I...,"Organization inf"" Vol. No. January-February is...","Organization inf"" Vol. No. January-February is...",organization inf vol january february issn ei...,0.424168,0.690747,0.709491
7,610,"[['CEO', 'Narcissism,', 'Audience', 'Engagemen...",1.0,Management & Organizational Behavior,10.2307_43546650,"CEO Narcissism, Audience Engagement, and Organ...","CEO Narcissism, Audience Engagement and Organi...","CEO Narcissism, Audience Engagement and Organi...",ceo narcissism audience engagement organizati...,0.364441,0.648315,0.712160
8,916,"[['Organizational', 'Structure', 'and', 'the',...",1.0,Management & Organizational Behavior,10.2307_2392303,Organizational Structure and the Institutional...,Organizational Structure and the Institutional...,Organizational Structure and the Institutional...,organizational structure institutional enviro...,0.416715,0.652464,0.603216
9,644,"[['Cultures', 'of', 'Culture:', 'This', 'paper...",1.0,Management & Organizational Behavior,10.2307_2392854,"Cultures of Culture: Academics, Practitioners ...",Cultures of Culture: This paper presents metho...,Cultures of Culture: This paper presents metho...,cultures culture paper presents method assess...,0.409967,0.616115,0.691231
